# [Universal Dependencies와 BERT/RoBERTa 모델을 통한 고전 중국어 정보처리](http://www.han-character.education/home/index.php/2022/11/18/20221118/)

## 古典中国語RoBERTaの穴埋めゲーム

In [ ]:
!pip install transformers
from transformers import pipeline
fmp=pipeline("fill-mask","KoichiYasuoka/roberta-classical-chinese-base-char")
prd=fmp("勞[MASK]者治人")
print("\n".join("{:8} {:.3f}".format(t["token_str"],t["score"]) for t in prd))

## 系列ラベリングによる品詞付与・単語切り

In [ ]:
!pip install deplacy transformers
class SeqL(object):
  def __init__(self,bert):
    from transformers import pipeline
    self.tagger=pipeline(task="ner",model=bert)
  def __call__(self,text):
    w=[(t["start"],t["end"],t["entity"]) for t in self.tagger(text)]
    u="# text = "+text.replace("\n"," ")+"\n"
    for i,(s,e,p) in enumerate(w,1):
      m,q="_" if i<len(w) and e<w[i][0] else "SpaceAfter=No",p.split("|")
      f="_" if p.find("=")<0 else "|".join(t for t in q if t.find("=")>0)
      u+="\t".join([str(i),text[s:e],"_",q[0],"_",f,"_","_","_",m])+"\n"
    return u+"\n"
nlp=SeqL("KoichiYasuoka/roberta-classical-chinese-base-ud-goeswith")
doc=nlp("未聞好學者也")
import deplacy
deplacy.serve(doc,port=None)